# Яндекс.Музыка

## Этап 1. Получение и обзор данных

In [1]:
# импорт библиотеки 
import pandas as pd

In [2]:
# получение файла и сохранение в переменную
df = pd.read_csv('/Users/a1/Documents/Practicum/Projects/big_cities_music/yandex_music_project_dataset.csv')

In [3]:
# получение первых десяти строк таблицы
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получение общей информации о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


**Выводы**

Итак, в таблице семь столбцов. Тип данных во всех столбцах — object.

userID — идентификатор пользователя;
Track — название трека;
artist — имя исполнителя;
genre — название жанра;
City — город, в котором происходило прослушивание;
time — время, в которое пользователь слушал трек;
Day — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения. Помимо этого, в названиях колонок видны различные нарушения стиля (строчные/прописные буквы, пробелы, нет нижних подчеркиваний)

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Этап 2. Предобработка данных

In [5]:
# выведение на экран названия столбцов
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

В названиях стоблцом есть признаки плохого стиля. Перезапишем их.

In [6]:
# исправление названий колонок в таблице
df = df.rename(columns = {'  userID' : 'user_id', 'Track' : 'track', '  City  ' : 'city', 'Day' : 'day'})

Проверим результат. Для этого ещё раз выведем на экран названия столбцов:

In [7]:
# проверка результата
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Теперь проверим данные на наличие пропусков.

In [8]:
# подсчет количества пропущенных значений в таблице
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пустые значения показывают, что для некоторых треков доступна не вся информация. Причины могут быть разные: например, нет имени конкретного исполнителя. Возможны проблемы с записью данных.

Не все пропущенные значения влияют на исследование. Так в track и artist пропуски не важны для нашего исследования. Достаточно заменить их явными обозначениями (‘unknown’). Ситуация с genre сложнее, но в рамках данной работы мы поступим с этим столбцом таким же образом, хотя есть опасность изменить общую картину.

Заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создадим список `columns_to_replace`, переберем его элементы циклом `for` и для каждого столбца выполним замену пропущенных значений:

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [10]:
# подсчёт пропусков
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Необходимо установить наличие дубликатов. Удаляем найденные, если таковые будут, и снова проверяем таблицу.


In [11]:
# подсчёт явных дубликатов
df.duplicated().sum()

3826

In [12]:
# удаление явных дубликатов
df = df.drop_duplicates()

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Разобравшись с явными дубликатами, теперь мы можем перейти к анализу неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Сохраним список уникальных значений столбца с жанрами в переменной genres_list.

Объявим функцию find_genre() для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.

In [14]:
genre_list = df['genre'].unique()
genre_list

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb', 'hip',
       'jazz', 'postrock', 'latin', 'classical', 'metal', 'reggae',
       'tatar', 'blues', 'instrumental', 'rusrock', 'dnb', 'türk', 'post',
       'country', 'psychedelic', 'conjazz', 'indie', 'posthardcore',
       'local', 'avantgarde', 'punk', 'videogame', 'techno', 'house',
       'christmas', 'melodic', 'caucasian', 'reggaeton', 'soundtrack',
       'singer', 'ska', 'shanson', 'ambient', 'film', 'western', 'rap',
       'beats', "hard'n'heavy", 'progmetal', 'minimal', 'contemporary',
       'new', 'soul', 'holiday', 'german', 'tropical', 'fairytail',
       'spiritual', 'urban', 'gospel', 'nujazz', 'folkmetal', 'trance',
       'miscellaneous', 'anime', 'hardcore', 'progressive', 'chanson',
       'numetal', 'vocal', 'estrada', 'russian', 'classicmetal',
       'dubstep', 'club', 'deep', 'southern', 'black', 'folkrock',
       'fitness', '

In [15]:
def find_genre(genre_name):
    count = 0
    for i in genre_list:
        if i == genre_name:
            count += 1
    return count

Вызовем функцию find_genre() для поиска различных вариантов названия жанра "хип-хоп" в таблице.

Правильное название — hiphop. Поищем другие варианты:

hip
hop
hip-hop

In [16]:
# вызов функции find_genre() проверяется наличие варианта 'hip'
find_genre('hip')

1

In [17]:
# вызов функции find_genre() проверяется наличие варианта ‘hop’
find_genre('hop')

1

In [18]:
# вызов функции find_genre() проверяется наличие варианта 'hip-hop'
find_genre('hip-hop')

1

Чтобы очистить таблицу воспользуемся методом `replace()` с двумя аргументами: списком строк-дубликатов (включащем *hip*, *hop* и *hip-hop*) и строкой с правильным значением.

In [19]:
# Устранение неявных дубликатов
df['genre']= df['genre'].replace(['hip', 'hop', 'hip-hop'], 'hiphop')

Проверим результат. Выведем отсортированный список уникальных значений столбца `genre`:

In [20]:
# Проверка на неявные дубликаты
genre_list = df['genre'].unique()
genre_list

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'tatar', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'shanson', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'contemporary', 'new', 'soul', 'holiday', 'german',
       'tropical', 'fairytail', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'chanson', 'numetal', 'vocal', 'estrada', 'russian',
       'classicmetal', 'dubstep', 'club', 'deep', 'southern', 'black',
       'folkrock', 'fitness'

**Выводы**

В ходе предобработки данных были обнаружены три проблемы:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты (явные и неявные).

Мы исправили заголовки и убрали дубликаты, а пропущенные значения заменили на `'unknown'`. Мы не просто удаляем все пропущенные значения, а заполняем их, потому что это может быть важно для анализа. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

## Этап 3. Проверка гипотез

### Есть ли связь дня недели с количеством прослушанных композиций?

В начале исследовния была выдвинута гипотеза о том, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных в эти дни композиций с известным жанром, и сравниваем результаты.

Оценим активность пользователей в каждом городе. Сгруппируем данные по городу и вызовом метода *count()* подсчитываем композиции, для которых известен жанр.

In [21]:
# Подсчёт прослушиваний в каждом городе
track_list_city = df.groupby('city')['genre'].count()
track_list_city

city
Moscow              42741
Saint-Petersburg    18512
Name: genre, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитаем прослушивания в понедельник, среду и пятницу.

In [22]:
# Подсчёт прослушиваний в каждый из трёх дней
track_list_grouped_by_day = df.groupby('day')['city'].count()
track_list_grouped_by_day

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создаём функцию number_tracks(). Она принимает как параметры день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города по дням недели (понедельник, среда и пятница).

In [23]:
def number_tracks(day, city):
        track_list = df[df['day'] == day] 
        track_list = track_list[track_list['city'] == city]
        track_list_count = track_list['user_id'].count()
        return track_list_count

Вызовем `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [24]:
# количество прослушиваний в Москве по понедельникам
number_tracks_per_city_per_day = number_tracks('Monday', 'Moscow')
number_tracks_per_city_per_day

15740

In [25]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks_per_city_per_day = number_tracks('Monday', 'Saint-Petersburg')
number_tracks_per_city_per_day

5614

In [26]:
# количество прослушиваний в Москве по средам
number_tracks_per_city_per_day = number_tracks('Wednesday', 'Moscow')
number_tracks_per_city_per_day

11056

In [27]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks_per_city_per_day = number_tracks('Wednesday', 'Saint-Petersburg')
number_tracks_per_city_per_day

7003

In [28]:
# количество прослушиваний в Москве по пятницам
number_tracks_per_city_per_day = number_tracks('Friday', 'Moscow')
number_tracks_per_city_per_day

15945

In [29]:
# количество прослушиваний в Санкт-Петербурге по пятницамnumber_tracks_per_city_per_day = number_tracks('Monday', 'Moscow')
number_tracks_per_city_per_day = number_tracks('Friday', 'Saint-Petersburg')
number_tracks_per_city_per_day

5895

Создадим таблицу с полученными значениями.

In [30]:
# Таблица с результатами
info = pd.DataFrame(data=[['Moscow', 15740, 11056, 15945], ['Saint-Petersburg', 5614, 7003, 5895]], columns=['city', 'monday', 'wednesday', 'friday'])
info

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Выводы**

Данные говорят о следующем:

- В Москве прослушивания достигают максимума в понедельник и пятницу, а в среду снижаются.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу нашей гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраните таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [31]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']


In [32]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

Создадим функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция возвращает информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [33]:
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre_name и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравни результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [34]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
top_10_tracks = genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
top_10_tracks

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [35]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
top_10_tracks = genre_weekday(spb_general, 'Monday', '07:00', '11:00')
top_10_tracks

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [36]:
# вызов функции для вечера пятницы в Москве
top_10_tracks = genre_weekday(moscow_general, 'Friday', '17:00', '23:00')
top_10_tracks

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [37]:
# вызов функции для вечера пятницы в Петербурге
top_10_tracks = genre_weekday(spb_general, 'Friday', '17:00', '23:00')
top_10_tracks

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

    **Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Последняя гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`. Затем отсортируем результат в порядке убывания и сохраним его в таблице `moscow_genres`.

In [38]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [39]:
# просмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторим операции для Петербурга.

In [40]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

In [41]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

## Этап 4. Результаты исследования

Мы проверили три гипотезы и получили следующие выводы:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.

Эта гипотеза **полностью подтвердилась**.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам: в Москве слушают музыку жанра “world”, а в Петербурге — джаз и классику.

Таким образом, вторая гипотеза **частично подтвердилась**. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза **не подтвердилась**. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.